In [107]:
import sys

import csv
import json
import time
import datetime
from ast import literal_eval
import pandas as pd
import numpy as np
#import geohash
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import matplotlib.pyplot as plt
from pyspark.sql.functions import mean, min, max, sum
from pyspark.sql import functions as func
import glob
import matplotlib.colors
from collections import Counter

## Globals

In [108]:
# folder path
resultsdir= ""

# attributes
fn="_all_counts.csv"
fn_age="_age.csv"
fn_gender="_gender.csv"

# load targetzones(NRW)
tz=pd.read_csv(resultsdir + "170830_munich_NW_MTC_unambiguous.csv")
tz_list=map(int,list(tz["id"].values))

# load mcc list
mccs=pd.read_csv("MCCs.csv")

# Time mapping
time_mapping={2:"2",3:"3",4:"4",5:"5",6:"6",7:"7",8:"8",9:"9",10:"10",11:"11",12:"12",13:"13",14:"14",15:"15",\
             16:"16",17:"17",18:"18",19:"19",20:"20",21:"21",22:"22",23:"23",0:"0+1",1:"1+1"}
hh=["2","3","4","5","6","7","8","9","10","11","12","13","14","15",\
             "16","17","18","19","20","21","22","23","0+1","1+1"]

## Input

In [109]:
# Cluster to analyze
weekday_cluster = "FRI"

# This is the number of days that were aggregated
number_of_days=4


## All Counts

In [110]:
#save for customer

nf=pd.read_csv(weekday_cluster+fn)
nf=nf[nf["dominant_zone"].isin(tz_list)]
nf=nf.sort_values("count",ascending=0)
nf=nf[nf["count"]>0]
sum_counts=np.sum(nf["count"].values)

print(sum_counts)


nf["hour"]=nf.apply(lambda row:(time_mapping[int(row["time"].split(':')[0])]),axis=1)
nf["count"]=nf.apply(lambda row:int(np.round(row["count"]/number_of_days,0)),axis=1)
nf=nf.rename(columns={"dominant_zone":"zone"})
nf=nf.drop(["time"],1)
nf=nf.sort_values("count",ascending=0)
sum_counts=np.sum(nf["count"].values)
print("Anzahl groups: " + str(len(np.unique(nf["zone"].values))))
print("Anzahl Kombinationen: " + str(len(nf)))
print(sum_counts)
nf["cluster"] = weekday_cluster
#save as a flat table
nf.to_csv(resultsdir+"Munich_NW_"+weekday_cluster+"_all_counts.csv",index=False)

#save as matrix
nf=nf.reset_index()
res=pd.DataFrame(index=list(np.unique(nf["zone"].values)),columns=hh)
for l in range(0,len(nf)):
    res.loc[nf["zone"][l],nf["hour"][l]]=nf["count"][l]
res=res.fillna(0)
res.to_csv(resultsdir+"Munich_NW_"+weekday_cluster+"_all_counts_matrix.csv")



151255817.118
Anzahl groups: 174
Anzahl Kombinationen: 4176
13750529


In [86]:
#age 

nf=pd.read_csv(resultsdir+weekday_cluster+fn_age)
nf=nf[nf["dominant_zone"].isin(tz_list)]
print(sum_counts)

nf["hour"]=nf.apply(lambda row:(time_mapping[int(row["time"].split(':')[0])]),axis=1)
nf["count"]=nf.apply(lambda row:int(np.round(row["count"]/number_of_days,0)),axis=1)
nf=nf.rename(columns={"dominant_zone":"zone"})
nf=nf[nf["count"]>0]
sum_counts=np.sum(nf["count"].values)
print(sum_counts)

nf["cluster"] = weekday_cluster
nf=nf[nf["age"].isin([1,2,3,4,5,6])]
nf=nf.reset_index()
nf["sum_age_info"]=nf.groupby(["zone","hour"])["count"].transform(np.sum)
nf["share"]=nf.apply(lambda row:row["count"]/float(row.sum_age_info),axis=1)

nf=nf.drop(["time","sum_age_info","count","index"],1)
nf.to_csv(resultsdir+"Munich_NW_"+weekday_cluster+"_age.csv",index=False)




11542848
11541661


In [90]:

nf=pd.read_csv(resultsdir+weekday_cluster+fn_gender)
nf=nf[nf["dominant_zone"].isin(tz_list)]
nf["hour"]=nf.apply(lambda row:(time_mapping[int(row["time"].split(':')[0])]),axis=1)
nf["count"]=nf.apply(lambda row:int(np.round(row["count"]/number_of_days,0)),axis=1)
nf=nf.rename(columns={"dominant_zone":"zone"})
nf=nf[nf["count"]>0]
sum_counts=np.sum(nf["count"].values)
print(sum_counts)

nf["cluster"] = weekday_cluster
nf=nf[nf["gender"].isin([1,2])]
nf=nf.reset_index()
nf["sum_gender_info"]=nf.groupby(["zone","hour"])["count"].transform(np.sum)
nf["share"]=nf.apply(lambda row:row["count"]/float(row.sum_gender_info),axis=1)

nf=nf.drop(["time","sum_gender_info","count","index"],1)
nf.to_csv(resultsdir+"Munich_NW_"+weekday_cluster+"_gender.csv",index=False)


11545768
